In [ ]:
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
# Load the image
image_path = 'RESISTORS-2.png'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Convert to HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define color range for segmentation
color_lower_bound = np.array([0, 0, 0])
color_upper_bound = np.array([255, 255, 255])

# Threshold the HSV image to get only the resistors colors
mask = cv2.inRange(hsv_image, color_lower_bound, color_upper_bound)

# Label connected components
label_image = measure.label(mask, background=0)
# Visualize the segmentation
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(label_image, cmap='jet')
plt.title('Segmented Image')
plt.axis('off')

plt.show()


In [ ]:
# Calculate the properties of each connected component
properties = measure.regionprops(label_image)

# Save the cropped resistor images
for idx, prop in enumerate(properties):
    if prop.area > 100:  # Filter out small areas
        # Create a mask for the component
        minr, minc, maxr, maxc = prop.bbox
        component_mask = (label_image == prop.label).astype(np.uint8) * 255
        cropped_mask = component_mask[minr:maxr, minc:maxc]

        # Mask the image to extract the resistor
        cropped_resistor = image[minr:maxr, minc:maxc]
        masked_resistor = cv2.bitwise_and(cropped_resistor, cropped_resistor, mask=cropped_mask)

        # Save the masked resistor image
        file_name = f'resistor_{idx+1}.png'
        cv2.imwrite(file_name, masked_resistor)
        print(f'Saved: {file_name}')
